In [15]:
# Gary's scintillation rate and # of hit SiPMs analysis code

# Import Libraries
import sys
import numpy as np

# from ana.SiPMPulses import SiPMPulses
ana_path = "../LAr10Ana/"
sys.path.insert(0, ana_path)
from ana import SiPMPulses
import importlib

importlib.reload(SiPMPulses)

# Functions to run code

# FFT cuts based on FFT shape
# Good waveforms should have max FFT near 0 MHz
def FFT_frequency_filtering(waveforms, dt, freq_cutoff_hz):

    filtered_waveforms = waveforms.copy()
    
    # Number of samples in waveform
    N = waveforms.shape[-1]

    # Compute the FFT values and frequencies
    fft_vals = np.abs(np.fft.fft(waveforms, axis=-1))
    fft_freq = np.abs(np.fft.fftfreq(N, d=dt))

    # Find where the maximum FFT val occurs
    max_amp_indices = np.argmax(fft_vals[:, :, 1:], axis=-1)

    # Don't need the fft_vals anymore
    del fft_vals

    # Get the frequency a which the max FFT val occurs
    peak_frequencies_hz = fft_freq[max_amp_indices]

    # Create a mask for noisy waveforms
    bad_channel_mask = peak_frequencies_hz >= freq_cutoff_hz

    # If it's a noisy waveform, replace it with zeros.
    filtered_waveforms[bad_channel_mask] = 0.
    return filtered_waveforms


# Computing the signal strength of waveforms
def calculate_signal_strength(fft_filtered_waveforms):

    # Compute the baseline of each waveform
    baseline = np.mean(fft_filtered_waveforms, axis=2)

    # Calculate signal strength
    signalPeak = np.min(fft_filtered_waveforms, axis=2) - baseline
    signalDroop = np.max(fft_filtered_waveforms, axis=2) - baseline
    signalStrength = signalDroop + signalPeak

    return signalStrength

# Simple function to unwrap CAEN timestamps
def unwrap_caen_timestamp(ts, max_ts):
    ts = np.asarray(ts, dtype=np.int64)

    # Detect rollovers
    rollovers = np.diff(ts) < 0

    # Cumulative count of rollovers
    rollover_count = np.concatenate([[0], np.cumsum(rollovers)])
    return ts + rollover_count * max_ts

# Main function to compute SiPMs hit per each file
def ScintillationRateAnalysis(ev):
    print(f"Scintillation Rate Analysis: Processing {ev["event_info"]["run_id"][0]} ev {ev["event_info"]["event_id"][0]}")
    output = {}

    scint = ev["scintillation"]
    # Load the waveforms
    waveforms = scint['Waveforms']

    # load other data which may be important
    sample_rate = 62.5e6
    decimation = ev['run_control']['caen']['global']['decimation']
    scint_timestamps = unwrap_caen_timestamp(scint['TriggerTimeTag'], 2**31)
    livetime = scint_timestamps[-1] * 8e-9  # timestmap is 8 ns

    if decimation == 0: decimation = 1
    dt = 1 / (sample_rate * decimation)        

    # Denoise waveforms and collect signal strength
    fft_filtered_waveforms = FFT_frequency_filtering(
        waveforms=waveforms, 
        dt=dt, 
        freq_cutoff_hz = 0.6 # MHz (Still rough guess, works well)
    )
    
    signal_strength = calculate_signal_strength(fft_filtered_waveforms)

    # Find all signals with strength < -10 (should be good pulses).
    signal_strength_limit = -10.
    mask = signal_strength < signal_strength_limit

    # Sum the total number of hits in the event
    NHists = np.sum(mask, axis=1).astype(np.uint8)
    output["n_hits"] = NHists

    # convert boolean mask array to uint32
    weights = 2**np.arange(32, dtype=np.uint32)
    output["good_waveforms_mask"] = mask.dot(weights)

    return output

In [16]:
oldBackgroundFiles = [
    "/exp/e961/data/SBC-25-daqdata/20251024_5.tar", # Background, 10/24, LAr, 51V bias, 1800 trig. thresh
    "/exp/e961/data/SBC-25-daqdata/20251024_7.tar", # Background, 10/24, LAr, 51V bias, 1900 trig. thresh
    "/exp/e961/data/SBC-25-daqdata/20251027_0.tar" # Background, 10/27, LAr, 51V bias, 1900 trig. thresh
]

backgroundFiles = [
    "/exp/e961/data/SBC-25-daqdata/20251107_28.tar", # Background 11/07, LAr, 51V bias, 4 decimation, 2050 CAEN threshold
    "/exp/e961/data/SBC-25-daqdata/20251112_14.tar", #Background, 51V bias, 4 decimation, 2050 CAEN threshold
    "/exp/e961/data/SBC-25-daqdata/20251118_5.tar", #Background, 51V bias, 4 decimation, 2050 CAEN threshold
    "/exp/e961/data/SBC-25-daqdata/20251118_6.tar", #Background, 51V bias, 4 decimation, 2050 CAEN threshold
    "/exp/e961/data/SBC-25-daqdata/20251124_10.tar", #Background, 51V bias, 4 decimation, 2050 CAEN threshold
    "/exp/e961/data/SBC-25-daqdata/20251203_5.tar", #Background, 51V bias, 4 decimation, 2050 CAEN threshold
    "/exp/e961/data/SBC-25-daqdata/20251209_0.tar",	#Background, 51V bias, 4 decimation, 2050 CAEN threshold
]

pulserCalibrationFiles = [
    "/exp/e961/data/SBC-25-daqdata/20251107_29.tar", # 51V bias, 4 decimation, 2050 CAEN threshold, 3V pulser
    "/exp/e961/data/SBC-25-daqdata/20251107_30.tar" #51V bias, 4 decimation, 2050 CAEN threshold, 10V pulser
]

Ba_133_calibrationFiles = [
    "/exp/e961/data/SBC-25-daqdata/20251123_4.tar", #Barium Source, 51V bias, 4 decimation, 2050 CAEN threshold
    "/exp/e961/data/SBC-25-daqdata/20251123_5.tar", #Barium Source, 51V bias, 4 decimation, 2050 CAEN threshold
    "/exp/e961/data/SBC-25-daqdata/20251119_4.tar" # Barium Source 51V bias, 4 decimation, 2050 CAEN threshold
]

Cs_137_calibrationFiles = [
    "/exp/e961/data/SBC-25-daqdata/20251124_13.tar", #Low Activity Cesium, 51V bias, 4 decimation, 1900 CAEN threshold
    "/exp/e961/data/SBC-25-daqdata/20251124_14.tar", #Low Activity Cesium, 51V bias, 4 decimation, 1850 CAEN threshold
    "/exp/e961/data/SBC-25-daqdata/20251201_2.tar", #Low Activity Cesium, 51V bias, 4 decimation, 1850 CAEN threshold
    "/exp/e961/data/SBC-25-daqdata/20251203_4.tar" #Low Activity Cesium, 51V bias, 4 decimation, 1850 CAEN threshold
]

Th_228_calibrationFiles = [
    "/exp/e961/data/SBC-25-daqdata/20251120_7.tar", # Thorium Source + DD generator, 51V bias, 4 decimation, 1900 CAEN threshold
    "/exp/e961/data/SBC-25-daqdata/20251120_8.tar" #Thorium Source, 51V bias, 4 decimation, 1900 CAEN threshold. DD Generator OFF
]

DD_files = [
    "/exp/e961/data/SBC-25-daqdata/20251121_4.tar" #DD Generator ON, 51V bias, 4 decimation, 2050 CAEN threshold
]

# from GetEvent import GetEvent
ana_path = "../LAr10Ana/"
sys.path.insert(0, ana_path)
import GetEvent
from sbcbinaryformat import Streamer, Writer
import os
importlib.reload(GetEvent)

# Compute the scintillation rates
background_ev =  GetEvent.GetEvent(backgroundFiles[0], 0, "run_control", "run_info", "event_info", "scintillation", 
                            strictMode=False, lazy_load_scintillation=False)
out = ScintillationRateAnalysis(background_ev)
print(out)

# Code to save to binary file.

# map numpy dtype.str names to those expected by sbcbinaryformat
def dname(s):
    s = s[1:] # remove leading carat
    if s == "f4": s = "f" # set float name
    if s == "f8": s = "d" # set double name

    return s

Scintillation Rate Analysis: Processing 20251107_28 ev 0
{'n_hits': array([1, 0, 0, ..., 1, 2, 0], shape=(68181,), dtype=uint8), 'good_waveforms_mask': array([      512,         0,         0, ..., 134217728,  33556480,
               0], shape=(68181,), dtype=uint32)}


In [14]:
column_names = list(out.keys())
dtypes = []
for c in column_names:
    val = out[c]
    # Convert to numpy array if it isn't already
    if not isinstance(val, np.ndarray):
        val = np.array(val)
    dtypes.append(dname(val.dtype.str))

# squeeze sizes
sizes = [list(np.squeeze(out[c]).shape) for c in column_names]
# set default
sizes = [s if len(s) else [1] for s in sizes]
# for outputs with a sub-event number, fix the sizes
sizes = [s[1:] if len(s) > 1 else s for s in sizes]

print(column_names, dtypes, sizes)

writer = Writer(os.path.join(os.getcwd(), f"test_scint_rate.sbc"), column_names, dtypes, sizes)
writer.write(out)

print("Wrote test_scint_rate.sbc")
data = Streamer("test_scint_rate.sbc").to_dict()
print(data)

['n_hits', 'good_event_mask'] ['u1', 'u4'] [[68181], [68181]]
Wrote test_scint_rate.sbc
OrderedDict({'n_hits': array([[1, 0, 0, ..., 1, 2, 0]], shape=(1, 68181), dtype=uint8), 'good_event_mask': array([[      512,         0,         0, ..., 134217728,  33556480,
                0]], shape=(1, 68181), dtype=uint32)})


In [24]:
data = GetEvent.GetEvent("/exp/e961/data/SBC-25-daqdata/20251020_5.tar", 0, "run_control", "run_info", "event_info", "scintillation", 
                            strictMode=False, lazy_load_scintillation=False)
data["scintillation"]["length"]

218992